In [1]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from ultralytics import YOLO

def put_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.8, color=(255, 255, 255), thickness=2, bg_color=(0, 0, 0)):
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    x, y = position
    cv2.rectangle(image, (x, y - text_height - 10), (x + text_width, y + baseline), bg_color, -1)  # Draw background rectangle
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def process_frame(frame, model, scaler_X, encoder, modelYolo):
    alto_original, ancho_original = frame.shape[:2]
    nuevo_alto = 1100  # Nueva altura deseada
    nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
    frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = modelYolo(image)

    keypoints = results[0].keypoints.data.numpy().tolist()[0]

    if len(keypoints) == 17:
        # Extraer ángulos y distancias relevantes
        left_shoulder = keypoints[5]
        right_shoulder = keypoints[6]
        left_elbow = keypoints[7]
        right_elbow = keypoints[8]
        left_wrist = keypoints[9]
        right_wrist = keypoints[10]
        left_hip = keypoints[11]
        right_hip = keypoints[12]
        left_knee = keypoints[13]
        right_knee = keypoints[14]
        left_ankle = keypoints[15]
        right_ankle = keypoints[16]

        # Ejemplo de ángulos
        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
        right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        features = np.array(keypoints).flatten().tolist()

        X = scaler_X.transform(np.array(features).reshape(1, -1))

        predictions = model.predict(X)

        predicciones_continuas = predictions[0][0]
        prediccion_clase = encoder.inverse_transform(np.argmax(predictions[1], axis=1))

        features_video = [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle]

        porcentaje_diferencia = 0.25
        caracteristicas = ["Angulo del codo izquierdo", "Angulo del codo derecho", "Angulo del hombro izquierdo", "Angulo del hombro derecho", "Angulo de la rodilla izquierda", "Angulo de la rodilla derecha"]

        array1 = np.array(predicciones_continuas)
        array2 = np.array(features_video)
        relacion = np.abs(1 - (array1 / array2))
        errores = [caracteristicas[i] for i in range(len(relacion)) if porcentaje_diferencia < relacion[i] < (1 - porcentaje_diferencia)]

        annotated_image = results[0].plot()
  
        clase_predicha = prediccion_clase[0]
        porcentaje_prediccion = np.max(predictions[1]) * 100
        texto_clasificacion = f'Clasificacion: {clase_predicha} - Probabilidad: {porcentaje_prediccion:.2f}%'
        put_text_with_background(annotated_image, texto_clasificacion, (50, 70), font_scale=1, bg_color=(0, 0, 0))

        # if len(errores) > 0:
        #     for i, error in enumerate(errores):
        #         desplazamiento_vertical = 120 + (i * 50)
        #         put_text_with_background(annotated_image, f'Error en : {error}', (50, desplazamiento_vertical), font_scale=1, bg_color=(0, 0, 0))

        return annotated_image

    return frame

def analyze_path(file_path, model, scaler_X, encoder, modelYolo):
    if file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        cap = cv2.VideoCapture(file_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            annotated_frame = process_frame(frame, model, scaler_X, encoder, modelYolo)
            cv2.imshow('Frame', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    elif file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        frame = cv2.imread(file_path)
        annotated_frame = process_frame(frame, model, scaler_X, encoder, modelYolo)
        rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('Image', rgb_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Formato de archivo no soportado")

# Cargar el modelo y las clases
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_dataset = os.path.join(ruta_raiz, 'Data Set')
ruta_archivo = os.path.join(ruta_dataset, 'test_video_3.mp4')  # Cambia esta ruta a la ruta del archivo que desees analizar
ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual, 'datos', 'scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')

# Cargar el modelo y scaler
model = load_model(ruta_modelo)
scaler_X = joblib.load(ruta_scaler)

classes = np.load(ruta_npy, allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes

modelYolo = YOLO("yolov8n-pose.pt")

# Realizar el análisis en la ruta dada
analyze_path(ruta_archivo, model, scaler_X, encoder, modelYolo)


0: 384x640 1 person, 108.5ms
Speed: 4.0ms preprocess, 108.5ms inference, 901.5ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step

0: 384x640 1 person, 78.6ms
Speed: 3.0ms preprocess, 78.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

0: 384x640 1 person, 68.0ms
Speed: 4.0ms preprocess, 68.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

0: 384x640 1 person, 64.5ms
Speed: 3.0ms preprocess, 64.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

0: 384x640 1 person, 63.5ms
Speed: 2.0ms preprocess, 63.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

0: 384x640 1 person, 76.5ms
Speed: 2.0ms preprocess, 76.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

0: 384x640 